In [3]:
!pwd

/home/kushal/TextSummarizer


In [2]:
import os
os.chdir('..')

In [4]:
from dataclasses import dataclass
from pathlib import Path

In [5]:
@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: str

In [6]:
from src.textSummarizer.constants import CONFIG_FILEPATH, PARAMS_FILEPATH
from src.textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, config_path = CONFIG_FILEPATH, params_path = PARAMS_FILEPATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        create_directories([self.config.artifacts_root])

    # def get_data_ingestion_config(self) -> DataIngestionConfig:
    #     config = self.config.data_ingestion
    #     create_directories([config.root_dir])
    #     data_ingestion_config = DataIngestionConfig(root_dir = config.root_dir, 
    #                                                source_URL = config.source_URL,
    #                                                local_data_file = config.local_data_file,
    #                                                unzip_dir = config.unzip_dir)
    #     return data_ingestion_config

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])
        data_tranformation_config = DataTransformationConfig(root_dir= config.root_dir,
                                                            data_path=config.data_path,
                                                            tokenizer_name=config.tokenizer_name)
        return data_tranformation_config

In [8]:
import os
from src.textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

/home/kushal/TextSummarizer/summarizer/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-02-16 10:48:21,803: INFO: config: PyTorch version 2.6.0 available.]


In [13]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_to_features(self, batch):
    # print(batch)
        input_tokens = self.tokenizer(batch['dialogue'], max_length=1024, truncation=True)
        output_tokens = self.tokenizer(batch['summary'], max_length=128, truncation=True)
    
        return {
            'input_ids' : input_tokens['input_ids'],
            'attention_mask': input_tokens['attention_mask'],
            'labels': output_tokens['input_ids']
        }

    def convert(self):
        dataset_samsun = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsun.map(self.convert_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, 'samsun_dataset'))

[2025-02-16 10:53:30,181: INFO: common: yaml file: config/config.yml loaded successfully]
[2025-02-16 10:53:30,183: INFO: common: yaml file: params.yml loaded successfully]
[2025-02-16 10:53:30,184: INFO: common: Directory created at: {path}]
[2025-02-16 10:53:30,185: INFO: common: Directory created at: {path}]


In [16]:
config = ConfigurationManager()
data_transformation_config = config.get_data_transformation_config()
data_transformation = DataTransformation(config = data_transformation_config)
data_transformation.convert()

Saving the dataset (1/1 shards): 100%|█████████████████████████████████████| 818/818 [00:00<00:00, 157267.17 examples/s]
